# Assignment4

### 1

For each of the shap_features in the files in the folders Random Forest Parameters and Gradient Boosting Parameters, find the 10 most common features and use those as the optimal features.

Find the P/L curves and compute annualized Sharp Ratios

In [1]:
!pip install --upgrade pandas dask

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.tree import DecisionTreeClassifier
import lightgbm
pd.set_option('use_inf_as_na', True)
from collections import Counter
import pickle

import optuna
from optuna.trial import Trial
import optuna.integration.lightgbm as lgb


from lightgbm import LGBMClassifier
from lightgbm import early_stopping
from lightgbm import log_evaluation
import sklearn.datasets
from sklearn.model_selection import train_test_split


from functools import partial

from sklearn.ensemble import RandomForestClassifier
import sklearn

optuna.logging.set_verbosity(optuna.logging.FATAL)
import warnings
warnings.filterwarnings("ignore")

/opt/anaconda3/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
print('The scikit-learn version is {}.'.format(sklearn.__version__))

The scikit-learn version is 1.2.2.


In [4]:
raw_data = pd.read_pickle('/Users/yiningqu/Desktop/研究生/33160 Machine Learning/Assignment2/dataset.pkl')
data = raw_data[raw_data['market_cap'] > 1000.0]
data = data.copy()
data.fillna(0.0,inplace=True)

In [5]:
def f(x):
    if x > 0.01:
        return 1
    elif x < -0.025:
        return -1
    else:
    
        return 0

In [6]:
data['rel_performance'] = data['pred_rel_return'].apply(f)
data.reset_index(inplace=True)
data.set_index('date',inplace=True)

In [7]:
from sklearn.preprocessing import MinMaxScaler,StandardScaler
scaler = StandardScaler()

In [8]:
start_dates = [pd.to_datetime('2001-01-01') + pd.DateOffset(months = 3*i) for i in range(57)]
end_dates = [d + pd.DateOffset(months = 36) for d in start_dates]

In [9]:
training_frames = [data.loc[d:d+pd.DateOffset(months = 36)] for d in start_dates]
valid_frames = [data.loc[d:d+pd.DateOffset(months = 3)] for d in end_dates]
test_frames = [data.loc[d+pd.DateOffset(months = 6):d+pd.DateOffset(months = 9)] for d in end_dates]

In [10]:
training_data = [d.reset_index().drop
                                 (['ticker','date',
                                   'next_period_return',
                                   'spy_next_period_return',
                                   'rel_performance','pred_rel_return',
                                  'return', 'cum_ret', 'spy_cum_ret'],axis=1) for d in training_frames]

In [11]:
valid_data = [d.reset_index().drop
                                 (['ticker','date',
                                   'next_period_return',
                                   'spy_next_period_return',
                                   'rel_performance','pred_rel_return',
                                  'return', 'cum_ret', 'spy_cum_ret'],axis=1) for d in valid_frames]

In [12]:
test_data = [d.reset_index().drop(['ticker','date',
                                   'next_period_return',
                                   'spy_next_period_return',
                                   'rel_performance','pred_rel_return',
                                  'return', 'cum_ret', 'spy_cum_ret'],axis=1) for d in test_frames]


In [13]:
training_labels = [d['rel_performance'].values for d in training_frames]
valid_labels = [d['rel_performance'].values for d in valid_frames] 

In [14]:
for i in range(len(start_dates)-1):
    float_vars = [x for x in training_data[i].columns if data[x].dtype == 'float64']
    scaler = StandardScaler()
    training_data[i] = training_data[i].copy()
    valid_data[i] = valid_data[i].copy()
    test_data[i] = test_data[i].copy()
    training_data[i][float_vars] = scaler.fit_transform(training_data[i][float_vars])
    valid_data[i][float_vars] = scaler.transform(valid_data[i][float_vars])
    test_data[i][float_vars] = scaler.transform(test_data[i][float_vars])
        
    

- Random Forest
  

In [15]:
def objective(trial:Trial,train=None,labels=None,val=None,val_labels=None,val_rets=None):

    rf_n_estimators = trial.suggest_int('n_estimators', 10,40,step=5)
    rf_max_features = trial.suggest_categorical('max_features',['sqrt','log2'])
    rf_min_samples_leaf = trial.suggest_int('min_samples_leaf',400,2400,step=400)
    rf_max_depth = trial.suggest_int('max_depth',5,25,step=5)
    
    rf_clf = RandomForestClassifier(n_estimators=rf_n_estimators,
                                    max_depth=rf_max_depth,
                                    min_samples_leaf=rf_min_samples_leaf,
                                    max_features=rf_max_features)
    rf_clf.fit(train,labels)
    preds = rf_clf.predict(val)
    profit = (preds * val_rets).sum()
 
    return profit

In [16]:
optimal_hyperparameters_rf = []

In [17]:
from functools import partial

In [18]:
for i in range(len(start_dates)-1):
    
    study = optuna.create_study(direction="maximize")
    
    study.optimize(partial(objective,
                           train=training_data[i],
                           labels=training_labels[i],
                           val=valid_data[i],
                           val_labels=valid_labels,
                           val_rets=valid_frames[i]['next_period_return']), n_trials=200,n_jobs=-1)
    
    optimal_hyperparameters_rf.append(study.best_params)  
    print(study.best_params)

{'n_estimators': 10, 'max_features': 'log2', 'min_samples_leaf': 400, 'max_depth': 10}
{'n_estimators': 25, 'max_features': 'sqrt', 'min_samples_leaf': 400, 'max_depth': 10}
{'n_estimators': 10, 'max_features': 'sqrt', 'min_samples_leaf': 1200, 'max_depth': 10}
{'n_estimators': 15, 'max_features': 'sqrt', 'min_samples_leaf': 400, 'max_depth': 10}
{'n_estimators': 10, 'max_features': 'sqrt', 'min_samples_leaf': 400, 'max_depth': 15}
{'n_estimators': 10, 'max_features': 'sqrt', 'min_samples_leaf': 1200, 'max_depth': 20}
{'n_estimators': 15, 'max_features': 'sqrt', 'min_samples_leaf': 400, 'max_depth': 5}
{'n_estimators': 25, 'max_features': 'sqrt', 'min_samples_leaf': 400, 'max_depth': 20}
{'n_estimators': 15, 'max_features': 'log2', 'min_samples_leaf': 2000, 'max_depth': 10}
{'n_estimators': 10, 'max_features': 'sqrt', 'min_samples_leaf': 400, 'max_depth': 10}
{'n_estimators': 25, 'max_features': 'sqrt', 'min_samples_leaf': 800, 'max_depth': 20}
{'n_estimators': 10, 'max_features': 'sqr

In [19]:
classifiers_rf = []

In [20]:
for hyp_par in optimal_hyperparameters_rf:
    rf_clf = RandomForestClassifier(**hyp_par)
    classifiers_rf.append(rf_clf)

In [21]:
import pickle
with open("/Users/yiningqu/Desktop/研究生/33160 Machine Learning/Assignment4/shap_features.pkl") as f:
    shap_features = pickle.load(f)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x80 in position 0: invalid start byte

In [ ]:
# find the top 10 most common features
shap_features_rf = shap_features
all_features_rf = [feature for sublist in shap_features_rf for feature in sublist]
count_feature_rf = Counter(all_features_rf)
optimal_feature_rf = count_feature_rf.most_common(10)
optimal_feature_rf = [feature[0] for feature in optimal_feature_rf[:10]]
optimal_feature_rf

In [ ]:
scalers_rf = [StandardScaler() for i in range(len(start_dates)-1)]  
        
opt_training_data = [pd.DataFrame(scalers_rf[i].fit_transform(training_frames[i][optimal_feature_rf].values),columns=optimal_feature_rf) for i in range(len(start_dates)-1)]
opt_valid_data = [pd.DataFrame(scalers_rf[i].fit_transform(training_frames[i][optimal_feature_rf].values),columns=optimal_feature_rf) for i in range(len(start_dates)-1)]
opt_test_data = [pd.DataFrame(scalers_rf[i].transform(test_frames[i][optimal_feature_rf].values),columns=optimal_feature_rf) for i in range(len(start_dates)-1)]

In [ ]:
x_rf = [1]
ret_rf = []

for i in range(len(start_dates)-1):
        
        classifiers_rf[i].fit(opt_training_data[i],training_labels[i])

        preds = classifiers_rf[i].predict(opt_test_data[i])
        profit_i = (preds*test_frames[i]['next_period_return']).sum()
        ret_rf.append(profit_i)
        num_names = len(opt_test_data[i])
        x_rf.append(x_rf[i] + (x_rf[i]/num_names)*profit_i)

plt.plot(x_rf)


In [ ]:
SPY = pd.read_pickle(r"SPY_cum_ret.pkl")
SPY = SPY.loc['2004-07-01':'2018-09-30']
SPY = SPY.resample('Q').ffill()
SPY['spy_cum_ret'] = (SPY['spy_cum_ret'] - SPY['spy_cum_ret'][0] + 1)
SPY['strategy'] = x_rf

In [ ]:
SPY = SPY.resample('Y').ffill()
SPY.plot()

In [ ]:
strategy_mean_ret = (SPY['strategy'] - 1).diff().mean()
strategy_std = (SPY['strategy'] - 1).diff().std()
strategy_sr = strategy_mean_ret/strategy_std
print('Annualized Strategy Sharpe Ratio with Random Forest: ',strategy_sr)

- Gradient Boosting

In [ ]:
def objective_2(trial:Trial,train=None,labels=None,val=None,val_labels=None,val_rets=None):
    
    params = {
        "objective": "multiclass",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "n_estimators":trial.suggest_int('n_estimators',10,100,step=10),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 10.0, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 5, 25),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf',400,2400,step=400),
        'learning_rate': trial.suggest_float('learning_rate',0.01,0.2,step=0.01)
    }
    
    gb_clf = LGBMClassifier(**params)
    gbm = gb_clf.fit(train,labels)
    preds = gbm.predict(val)
    profit = (preds*val_rets).sum()
    return profit

In [ ]:
optimal_hyperparameters_gb = []

In [ ]:
for i in range(len(start_dates)-1):
    
    study = optuna.create_study(direction="maximize")
    
    study.optimize(partial(objective_2,
                           train=training_data[i],
                           labels=training_labels[i],
                           val=valid_data[i],
                           val_labels=valid_labels[i],
                           val_rets=valid_frames[i]['next_period_return']), n_trials=25,n_jobs=-1)
    
    optimal_hyperparameters_gb.append(study.best_params)  
    print(study.best_params)

In [ ]:
classifiers_gb = []

for hyp_par in optimal_hyperparameters_gb:
    gb_clf = LGBMClassifier(**hyp_par)
    classifiers_gb.append(gb_clf)

In [ ]:
with open(r"shap_features_gb_clf_01.pkl",'rb') as f:
    shap_features = pickle.load(f)

In [ ]:
# find the top 10 most common features
shap_features_gb = shap_features
all_features_gb = [feature for sublist in shap_features_gb for feature in sublist]
count_feature_gb = Counter(all_features_gb)
optimal_feature_gb = count_feature_gb.most_common(10)
optimal_feature_gb = [feature[0] for feature in optimal_feature_gb[:10]]
optimal_feature_gb

In [ ]:
scalers_gb = [StandardScaler() for i in range(len(start_dates)-1)]  
        
opt_training_data = [pd.DataFrame(scalers_gb[i].fit_transform(training_frames[i][optimal_feature_gb].values),columns=optimal_feature_gb) for i in range(len(start_dates)-1)]
opt_valid_data = [pd.DataFrame(scalers_gb[i].fit_transform(training_frames[i][optimal_feature_gb].values),columns=optimal_feature_gb) for i in range(len(start_dates)-1)]
opt_test_data = [pd.DataFrame(scalers_gb[i].transform(test_frames[i][optimal_feature_gb].values),columns=optimal_feature_gb) for i in range(len(start_dates)-1)]


In [ ]:
x_gb = [1]
ret_gb = []

for i in range(len(start_dates)-1):
        
        classifiers_gb[i].fit(opt_training_data[i],training_labels[i])

        preds = classifiers_gb[i].predict(opt_test_data[i])
        profit_i = (preds*test_frames[i]['next_period_return']).sum()
        ret_gb.append(profit_i)
        num_names = len(opt_test_data[i])
        x_gb.append(x_gb[i] + (x_gb[i]/num_names)*profit_i)

In [ ]:
plt.plot(x_gb)

In [ ]:
SPY = pd.read_pickle(r"SPY_cum_ret.pkl")
SPY = SPY.loc['2004-07-01':'2018-09-30']
SPY = SPY.resample('Q').ffill()
SPY['spy_cum_ret'] = (SPY['spy_cum_ret'] - SPY['spy_cum_ret'][0] + 1)
SPY['strategy'] = x_gb

In [ ]:
SPY = SPY.resample('Y').ffill()
SPY.plot()

In [ ]:
strategy_mean_ret = (SPY['strategy'] - 1).diff().mean()
strategy_std = (SPY['strategy'] - 1).diff().std()
strategy_sr = strategy_mean_ret/strategy_std
print('Annualized Strategy Sharpe Ratio with Gradient Boosting: ',strategy_sr)

### **Question 2**:
Combine these strategies, by choosing weights on each strategy and try to find a combined strategy with the best Sharp Ratio. You can also try to include a short of the SPY by including SPY with a negative weight.

In [ ]:
SPY = pd.read_pickle(r"SPY_cum_ret.pkl")
SPY = SPY.loc['2004-07-01':'2018-09-30']
SPY = SPY.resample('Q').ffill()
SPY['spy_cum_ret'] = (SPY['spy_cum_ret'] - SPY['spy_cum_ret'][0] + 1)
SPY['strategy_rf'] = x_rf
SPY['strategy_gb'] = x_gb

In [ ]:
SPY = SPY.resample('Y').ffill()
SPY.plot()

Try 50% weight of Random Forest and 50% weight of Gradient Boosting

In [ ]:
# try 50% weight of random forest strategy and 50% weight of gradient boosting
SPY['rf50gb50'] = SPY['strategy_rf']*0.5+SPY['strategy_gb']*0.5

In [ ]:
rf50gb50_mean_ret = (SPY['rf50gb50'] - 1).diff().mean()
rf50gb50_std = (SPY['rf50gb50'] - 1).diff().std()
rf50gb50_sr = rf50gb50_mean_ret/rf50gb50_std
print('Annualized Strategy Sharpe Ratio with 50% weight of Random Forest an 50% weight of Gradient Boosting: ',rf50gb50_sr)

Try different combinations of weights

In [ ]:
num_simulations = 20
results = []

for _ in range(num_simulations):
    weight_rf = np.random.rand()  
    weight_gb = 1 - weight_rf  

    SPY['combination'] = (SPY['strategy_rf'] * weight_rf +
                          SPY['strategy_gb'] * weight_gb)

    diffs = (SPY['combination'] - 1).diff()
    annual_returns = diffs.mean()
    annual_std = diffs.std()
    sharpe_ratio = annual_returns / annual_std

    results.append(((weight_rf, weight_gb), sharpe_ratio))

for i, (combination, sharpe_ratio) in enumerate(results): 
        print(f"Weights (RF, GB): {combination}, Sharpe Ratio: {sharpe_ratio:.2f}")

In [ ]:
# find the best Sharpe Ratio among all simulations
best_combination, best_sharpe_ratio = max(results, key=lambda x: x[1])
print(f'Best Combination Weights (RF, GB): {best_combination}')
print(f'Best Sharpe Ratio: {best_sharpe_ratio}')